In [587]:
import pandas as pd
import numpy as np

In [573]:
#LETS MOVE ON TO THE NEXT DATASET
# Load Budget data
budget_df = pd.read_excel("Assignment-Budget.xlsx", sheet_name="Data")

In [574]:
print("Total records:", len(budget_df))

Total records: 889


In [575]:
# Assume the meaningful headers are on row 4 (index 3)
new_header = budget_df.iloc[3]

# Drop all rows above (and including) the header row, then reset index
budget_df = budget_df.iloc[4:].reset_index(drop=True)

# Set the new header
budget_df.columns = new_header

# Optional: clean column names by stripping extra spaces
budget_df.columns = budget_df.columns.str.strip()
print(budget_df.head())


3  NaN Customer Code Item Code    Business Channel     Branded/PL Item Group  \
0  NaN        761644     53672  Business A  Retail  Private Label   Crackers   
1  NaN        760394     53672  Business A  Retail  Private Label   Crackers   
2  NaN        793483     53672  Business A  Retail  Private Label   Crackers   
3  NaN        760401     53672  Business A  Retail  Private Label   Crackers   
4  NaN        793480     53672  Business A  Retail  Private Label   Crackers   

3 Jan '21 Feb '21 Mar '21  ...  Apr '21  May '21 Jun '21 Jul '21   Aug '21  \
0    3960  2851.2  3484.8  ...  9694.08        0       0       0  30697.92   
1  3326.4  3168.0  2851.2  ...        0        0       0       0  45239.04   
2  1425.6   792.0   950.4  ...        0        0       0       0  12925.44   
3  9028.8  4118.4  3484.8  ...  6462.72  24235.2       0       0         0   
4  8236.8       0  4118.4  ...  6462.72        0       0       0   32313.6   

3     Sep '21      Oct '21     Nov '21     Dec '21

In [577]:
# Identify the column groups
    # First 6 columns are identifiers
identifier_cols = budget_df.columns[:6]
    
    # Find the 'a' separator column that separates Cases from Dollars
separator_col_idx = budget_df.columns.get_loc('a')
    
    # Cases columns come before the separator
cases_cols = budget_df.columns[6:separator_col_idx]
    
    # Dollars columns come after the separator, excluding the 'x' column at the end
amount_cols = budget_df.columns[separator_col_idx+1:-1]
    
    # Create separate dataframes for cases and dollars
df_cases = pd.melt(
    budget_df,
    id_vars=identifier_cols,
    value_vars=cases_cols,
    var_name="Month",
    value_name="Cases"
)
    
df_dollars = pd.melt(
    budget_df,
    id_vars=identifier_cols,
    value_vars=amount_cols,
    var_name="Month",
    value_name="Dollars"
)

In [589]:
# Create a mapping from dollars month columns to cases month columns
month_mapping = {}
for i, dollars_col in enumerate(amount_cols):
    if i < len(cases_cols):
        month_mapping[dollars_col] = cases_cols[i]

# Apply the mapping to standardize month names
df_dollars["Month"] = df_dollars["Month"].map(month_mapping)

In [590]:
# Merge the dataframes
unpivoted_df = pd.merge(df_cases, df_dollars, on=list(identifier_cols) + ["Month"])

In [608]:
#Cleaning
unpivoted_df.shape

(9802, 10)

In [604]:
# Remove duplicate rows from the unpivoted DataFrame
unpivoted_df = unpivoted_df.drop_duplicates()

# Verify if duplicates are removed
remaining_duplicates = unpivoted_df.duplicated().sum()

# Print confirmation
print(f"Remaining duplicate rows after removal: {remaining_duplicates}")


Remaining duplicate rows after removal: 0


In [607]:
# Ensure columns exist before filtering
if "Cases" in unpivoted_df.columns and "Dollars" in unpivoted_df.columns:
    unpivoted_df = unpivoted_df[unpivoted_df["Cases"] != unpivoted_df["Dollars"]]

# Verify if rows were removed
print(f"Remaining rows after removal: {merged_budget.shape[0]}")


Remaining rows after removal: 0


In [601]:
# Merge the dataframes
unpivoted_df.head

<bound method NDFrame.head of       CustomerCode ProductCode    Business       Channel ProductCategory  \
0           761644       53672  Business A        Retail   Private Label   
1           761644       53672  Business A        Retail   Private Label   
2           760394       53672  Business A        Retail   Private Label   
3           760394       53672  Business A        Retail   Private Label   
4           793483       53672  Business A        Retail   Private Label   
...            ...         ...         ...           ...             ...   
42475       800224       50040  Business A  Food Service         Branded   
42476       900009       50015  Business A  Food Service         Branded   
42477       900009       50015  Business A  Food Service         Branded   
42478       900009       50040  Business A  Food Service         Branded   
42479       900009       50040  Business A  Food Service         Branded   

      ProductGroup    Month     Cases   Dollars  MonthNum

In [597]:
unpivoted_df.to_csv("test2.csv", index=False)

In [600]:
# Rename key columns for consistency
unpivoted_df = unpivoted_df.rename(columns={"Customer Code": "CustomerCode", "Item Code": "ProductCode", "Branded/PL": "ProductCategory","Item Group": "ProductGroup"})


In [630]:
# Step 1: Check for Missing Values - Important to kote that 3 customer codes were missing in og data 12*3 in the final 
missing_values = unpivoted_df.isnull().sum()
print(missing_values)

Year               0
MonthNum           0
Month              0
MonthDate          0
Business           0
Channel            0
CustomerCode       0
ProductCode        0
ProductCategory    0
ProductGroup       0
Cases              0
Dollars            0
dtype: int64


In [610]:
# Fill missing CustomerCodes using most frequent CustomerCode for the same ProductCode
unpivoted_df["CustomerCode"] = unpivoted_df.groupby("ProductCode")["CustomerCode"].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)

# If still missing, set to "Unknown"
unpivoted_df["CustomerCode"].fillna("Unknown", inplace=True)


/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_94808/3260781192.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  unpivoted_df["CustomerCode"] = unpivoted_df.groupby("ProductCode")["CustomerCode"].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)
/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_94808/3260781192.py:2: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  unpivoted_df["CustomerCode"] = unpivoted_df.groupby("ProductCode")["CustomerCode"].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)
/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_94808/3260781192.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [613]:
# Replace "Private Label" with "PL" in the ProductCategory column
unpivoted_df["ProductCategory"] = unpivoted_df["ProductCategory"].replace({"Private Label": "PL"})

# Verify the change
print(unpivoted_df["ProductCategory"].unique())


['PL' 'Branded']


/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_94808/4213767037.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpivoted_df["ProductCategory"] = unpivoted_df["ProductCategory"].replace({"Private Label": "PL"})


In [629]:
# Step 5: Verify Data Types & Convert Where Necessary
data_types_before = unpivoted_df.dtypes
print(data_types_before)

Year                       object
MonthNum                    int64
Month                      object
MonthDate          datetime64[ns]
Business                   object
Channel                    object
CustomerCode               object
ProductCode                object
ProductCategory            object
ProductGroup               object
Cases                      object
Dollars                    object
dtype: object


In [628]:
unpivoted_df.head

<bound method NDFrame.head of        Year  MonthNum    Month  MonthDate    Business       Channel  \
1      2021         1  Jan '21 2021-01-01  Business A        Retail   
3      2021         1  Jan '21 2021-01-01  Business A        Retail   
5      2021         1  Jan '21 2021-01-01  Business A        Retail   
7      2021         1  Jan '21 2021-01-01  Business A        Retail   
9      2021         1  Jan '21 2021-01-01  Business A        Retail   
...     ...       ...      ...        ...         ...           ...   
42470  2021        12  Dec '21 2021-12-01  Business A  Food Service   
42472  2021        12  Dec '21 2021-12-01  Business A  Food Service   
42474  2021        12  Dec '21 2021-12-01  Business A  Food Service   
42476  2021        12  Dec '21 2021-12-01  Business A  Food Service   
42478  2021        12  Dec '21 2021-12-01  Business A  Food Service   

      CustomerCode ProductCode ProductCategory ProductGroup     Cases  \
1           761644       53672              

In [624]:
# Cleaner way to extract the year
unpivoted_df['Year'] = unpivoted_df['Month'].apply(lambda x: "20" + x.split("'")[1].strip())

# Extract Month Name (just the month part like "Jan")
unpivoted_df['MonthName'] = unpivoted_df['Month'].apply(lambda x: x.split("'")[0].strip())

# Create MonthDate as a proper date
unpivoted_df['MonthDate'] = pd.to_datetime(
    unpivoted_df['Year'].astype(str) + '-' + 
    unpivoted_df['MonthNum'].astype(str).str.zfill(2) + '-01'
)

/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_94808/3645102631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpivoted_df['Year'] = unpivoted_df['Month'].apply(lambda x: "20" + x.split("'")[1].strip())
/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_94808/3645102631.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpivoted_df['MonthName'] = unpivoted_df['Month'].apply(lambda x: x.split("'")[0].strip())
/var/folders/r8/nhc1hq510sgf0m_19lwsv4qh0000gn/T/ipykernel_94808/3645102

In [627]:
# Define the new column order
new_column_order = [
    "Year","MonthNum", "Month","MonthDate", "Business", "Channel","CustomerCode","ProductCode",
    "ProductCategory", "ProductGroup","Cases","Dollars"]

# Reorder the columns
unpivoted_df = unpivoted_df[new_column_order]

# Verify the new order
print(unpivoted_df.head())

   Year  MonthNum    Month  MonthDate    Business Channel CustomerCode  \
1  2021         1  Jan '21 2021-01-01  Business A  Retail       761644   
3  2021         1  Jan '21 2021-01-01  Business A  Retail       760394   
5  2021         1  Jan '21 2021-01-01  Business A  Retail       793483   
7  2021         1  Jan '21 2021-01-01  Business A  Retail       760401   
9  2021         1  Jan '21 2021-01-01  Business A  Retail       793480   

  ProductCode ProductCategory ProductGroup   Cases   Dollars  
1       53672              PL     Crackers    3960     40392  
3       53672              PL     Crackers  3326.4  33929.28  
5       53672              PL     Crackers  1425.6  14541.12  
7       53672              PL     Crackers  9028.8  92093.76  
9       53672              PL     Crackers  8236.8  84015.36  


In [ ]:
#Changing data types

In [632]:
print(unpivoted_df.head())

   Year  MonthNumber    Month  MonthDate    Business Channel CustomerCode  \
1  2021            1  Jan '21 2021-01-01  Business A  Retail       761644   
3  2021            1  Jan '21 2021-01-01  Business A  Retail       760394   
5  2021            1  Jan '21 2021-01-01  Business A  Retail       793483   
7  2021            1  Jan '21 2021-01-01  Business A  Retail       760401   
9  2021            1  Jan '21 2021-01-01  Business A  Retail       793480   

  ProductCode ProductCategory ProductGroup Budgeted_Boxes Budgeted_Revenue  
1       53672              PL     Crackers           3960            40392  
3       53672              PL     Crackers         3326.4         33929.28  
5       53672              PL     Crackers         1425.6         14541.12  
7       53672              PL     Crackers         9028.8         92093.76  
9       53672              PL     Crackers         8236.8         84015.36  


In [631]:
# Rename specific columns using a dictionary
unpivoted_df = unpivoted_df.rename(columns={
    'MonthNum': 'MonthNumber',
    'Cases': 'Budgeted_Boxes',
    'Dollars': 'Budgeted_Revenue',
})

In [633]:
# Define optimal data types for each column
data_type_mapping = {
    "CustomerCode": "category",  # Categorical data
    "ProductCode": "category",  # Categorical data
    "ProductCategory": "category",  # Branded vs. Private Label
    "Budgeted_Boxes": "int32",  # Integer values for quantity
    "Budgeted_Revenue": "float32",
    "Business": "category",
    "ProductGroup": "category", 
    "Channel": "category",
    "Month":"string",
    "Year":"int32",
    "MonthNumber":"int32",
    # Monetary values
    
}

# Apply data type conversions
for col, dtype in data_type_mapping.items():
    if col in unpivoted_df.columns:
        unpivoted_df[col] = unpivoted_df[col].astype(dtype, errors="ignore")

# Verify updated data types
print(unpivoted_df.dtypes)


Year                         int32
MonthNumber                  int32
Month               string[python]
MonthDate           datetime64[ns]
Business                  category
Channel                   category
CustomerCode              category
ProductCode               category
ProductCategory           category
ProductGroup              category
Budgeted_Boxes               int32
Budgeted_Revenue           float32
dtype: object


In [634]:
unpivoted_df.to_csv("Fact_Budget.csv", index=False)